In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [262]:
df= pd.read_csv(r'C:\Users\Malik\Desktop\Bengaluru_House_Data.csv')

In [263]:
df= df.drop(['area_type', 'society', 'balcony', 'availability'], axis= 'columns')

In [264]:
df.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [265]:
df=df.dropna()

In [266]:
len(df['location'].unique())   # unique locations

1304

In [267]:
df['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [268]:
len(df['size'].unique())

31

In [269]:
df['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [270]:
len(df['total_sqft'].unique())

2067

In [271]:
df['bath'].unique()

array([ 2.,  5.,  3.,  4.,  6.,  1.,  9.,  8.,  7., 11., 10., 14., 27.,
       12., 16., 40., 15., 13., 18.])

In [272]:
len(df['bath'].unique())

19

In [273]:
df['BHK'] = df['size'].apply(lambda x : int(x.split(' ')[0]))

In [274]:
df = df.drop('size', axis='columns')

In [275]:
df.head(3)

,location,total_sqft,bath,price,BHK
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3


In [276]:
df['total_sqft'].str.len().value_counts()

4     11095
3      1840
11       88
7        59
9        48
5        31
6        29
10       13
12       11
13        7
15        7
14        6
2         5
8         3
17        2
1         2
Name: total_sqft, dtype: int64

In [277]:
df.loc[df['total_sqft'].str.len()==17]

,location,total_sqft,bath,price,BHK
2228,KR Puram,1005.03 - 1252.49,3.0,47.405,3
3174,Jakkur,2249.81 - 4112.19,4.0,260.500,4


In [278]:
df['area1']= df['total_sqft'].apply(lambda x: ( float(x.split('-')[0]) + float(x.split('-')[1]) )/2 if len(x.split('-')) ==2 else x)

In [279]:
df['area1'].str.len().value_counts()

4.0     11095
3.0      1840
7.0        59
5.0        31
6.0        29
12.0       11
13.0        7
14.0        6
9.0         6
15.0        6
2.0         5
8.0         3
1.0         2
11.0        2
Name: area1, dtype: int64

In [280]:
# verify string presence in column
df['area1'].str.contains('\D', regex=True).sum()

133

In [281]:
# check dataypes of column
df.dtypes

location       object
total_sqft     object
bath          float64
price         float64
BHK             int64
area1          object
dtype: object

In [282]:
#extract numbers from string
df['area2']= df['area1'].str.extract('(\d+)')

In [283]:
df['area2'].str.len().value_counts()

4.0    11183
3.0     1866
5.0       27
2.0       16
1.0       10
Name: area2, dtype: int64

In [284]:
#price per square ft
df['price/sqft']= df['price'].astype(float)*100000 / df['area2'].astype(float)

In [285]:
#REMOVE WHITESPACE
df['location']=df['location'].apply(lambda x:x.strip())

In [286]:
#dimension reduction: location <10 datapoints to others

location_count = df.groupby('location')['location'].agg('count').sort_values(ascending=False)
other_location = location_count[location_count <= 10]
df['location']=df['location'].apply(lambda x:x.replace(x, 'other') if x in other_location else x)

In [287]:
#OHE
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [288]:
df['location1']= le.fit_transform(df['location'])

In [310]:
df

,location,total_sqft,bath,price,BHK,area1,area2,price/sqft,location1
0,Electronic City Phase II,1056,2.0,39.07,2,1056,1056,3699.810606,79
1,Chikka Tirupathi,2600,5.0,120.00,4,2600,2600,4615.384615,60
2,Uttarahalli,1440,2.0,62.00,3,1440,1440,4305.555556,226
3,Lingadheeranahalli,1521,3.0,95.00,3,1521,1521,6245.890861,159
4,Kothanur,1200,2.0,51.00,2,1200,1200,4250.000000,151
...,...,...,...,...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5,3453,3453,6689.834926,235
13316,other,3600,5.0,400.00,4,3600,3600,11111.111111,241
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2,1141,1141,5258.545136,194
13318,Padmanabhanagar,4689,4.0,488.00,4,4689,4689,10407.336319,185


In [311]:
x= df[['location1','area2', 'BHK']]
y= df['price']

In [314]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)

In [315]:
from sklearn.linear_model import LinearRegression
model= LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [316]:
model.score(x_test, y_test)

0.26726355942625346

In [348]:
def get_location(location):
    location1 = df['location1'].loc[df['location']== location].iloc[0]
    return location1

In [349]:
get_location('Uttarahalli')

226

In [352]:
def predict_price(location, area, BHK):
    location1 = df['location1'].loc[df['location']== location].iloc[0]
    return model.predict([[location1, area, BHK]])

In [353]:
predict_price('Uttarahalli', 3000, 3)

C:\Users\Malik\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([213.00314641])

In [354]:
import joblib

In [355]:
joblib.dump(model, 'house_model')

['house_model']

In [356]:
from flask import Flask

In [52]:
def floater(x):
    try:
        x_= float(x)
        return x_
    except:
        return x